In [1]:
import pandas as pd
df = pd.read_csv("spotify_millsongdata.csv")

In [2]:
df.head(100)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
95,ABBA,The Name Of The Game,/a/abba/the+name+of+the+game_20002863.html,"I've seen you twice, in a short time \r\nOnly..."
96,ABBA,The Piper,/a/abba/the+piper_20002946.html,They came from the hills \r\nAnd they came fr...
97,ABBA,The Visitors,/a/abba/the+visitors_20002864.html,I hear the doorbell ring and suddenly the pani...
98,ABBA,The Way Old Friends Do,/a/abba/the+way+old+friends+do_20002573.html,You and I can share the silence \r\nFinding c...


In [3]:
df.shape


(57650, 4)

In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
print(df.head(0))

Empty DataFrame
Columns: [artist, song, text]
Index: []


In [7]:
df.shape

(5000, 3)

Text Cleaning

In [8]:
df['text'].str.lower()

0       looks like we found ourselves  \r\nup against ...
1       clyde plays electric bass plays it with finess...
2       how 'bout a bit of soul?  \r\nsome integrity? ...
3       i'll never stop loving you  \r\nwhatever else ...
4       isn't it strange? feels like i'm lookin' in th...
                              ...                        
4995    am i  \r\nam i still tough enough?  \r\nfeels ...
4996    if it were just another day  \r\nyou know i mi...
4997    oh please release me, let me go  \r\nfor i jus...
4998    [chorus]  \r\ni gotta shake you off  \r\n'caus...
4999      \r\n'star fleet to scout ship, please give y...
Name: text, Length: 5000, dtype: object

In [9]:
df['text'] = df['text'].str.lower().replace(r'^\w+\s', ' ').replace(r'\n', ' ', regex =True)

In [10]:
print(df['text'])

0       looks like we found ourselves  \r up against a...
1       clyde plays electric bass plays it with finess...
2       how 'bout a bit of soul?  \r some integrity?  ...
3       i'll never stop loving you  \r whatever else i...
4       isn't it strange? feels like i'm lookin' in th...
                              ...                        
4995    am i  \r am i still tough enough?  \r feels li...
4996    if it were just another day  \r you know i mig...
4997    oh please release me, let me go  \r for i just...
4998    [chorus]  \r i gotta shake you off  \r 'cause ...
4999      \r 'star fleet to scout ship, please give yo...
Name: text, Length: 5000, dtype: object


In [11]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [13]:
import re
from nltk.stem import PorterStemmer

# Initialize the stemmer
stemmer = PorterStemmer()

# Function to tokenize and stem the text without using nltk's tokenizers
def token(txt):
    # Tokenize by splitting on non-alphabetic characters
    tokens = re.findall(r'\b\w+\b', txt.lower())  # Find all words
    stemmed_tokens = [stemmer.stem(word) for word in tokens]  # Stem each word
    return " ".join(stemmed_tokens)

# Test the function
print(token("you are beautiful, beauty"))


you are beauti beauti


In [14]:
df['text'] = df['text'].apply(lambda x: token(x))


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
TfidfVectorizer(analyzer='word', stop_words = 'english')

TfidfVectorizer(stop_words='english')

In [17]:
tfid = TfidfVectorizer()

# Step 2: Fit and transform the text data
matrix = tfid.fit_transform(df['text'])

# Display the shape of the TF-IDF matrix
print("TF-IDF Matrix Shape:", matrix.shape)

TF-IDF Matrix Shape: (5000, 17611)


In [18]:
similar = cosine_similarity(matrix)

In [19]:
similar[0]

array([1.        , 0.04975205, 0.06685132, ..., 0.11571053, 0.05527036,
       0.07286882])

In [20]:
df[df['song'] == 'The Name Of The Game'].index[0]

np.int64(3613)

reccomender function

In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similar[idx])),reverse=True, key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
print(recommendation("The Name Of The Game"))


["You're In My Heart", 'Think Of You', "It's Me", 'The Way You Talk To Me', 'If Your Love Cannot Be Moved', "Sometimes You Can't Make It On Your Own", 'Living For You', 'Every Time I Look At You', 'No You', 'I Want To Know What Love Is', "If Your Heart's Not In It", "I Mean I Don't Mean It", 'Feels Like', 'Lonely Heart', 'I Got You', 'Tell Me That You Love Me', 'Make My Life A Prayer To You', 'Say It', 'Now Is The Time', 'There Was A Time']


In [23]:
import pickle

In [24]:
pickle.dump(similar, open("similarity","wb"))

In [25]:
pickle.dump(df, open("df","wb"))